In [1]:
import duckdb
import json
import requests
import geopandas as gpd
import pandas as pd
import os

In [ ]:
countries_dict = {
  "ZA": "South_Africa",
  "BR": "Brazil",
  "KE": "Kenya",
  "YT": "Mayotte",
  "PK": "Pakistan",
  "CU": "Cuba",
  "DO": "Dominican_Republic",
  "SG": "Singapore",
  "SY": "Syria",
  "LU": "Luxembourg",
  "PA": "Panama",
  "TT": "Trinidad_And_Tobago",
  "ES": "Spain",
  "ZW": "Zimbabwe",
  "GN": "Guinea",
  "BA": "Bosnia_And_Herzegovina",
  "AW": "Aruba",
  "CW": "Curacao",
  "BS": "Bahamas",
  "IO": "British_Indian_Ocean_Territory",
  "JP": "Japan",
  "BL": "Saint_Barthelemy",
  "SX": "Sint_Maarten",
  "SB": "Solomon_Islands",
  "WF": "Wallis_And_Futuna",
  "MX": "Mexico",
  "KI": "Kiribati",
  "CI": "Ivory_Coast",
  "GM": "Gambia",
  "HT": "Haiti",
  "YE": "Yemen",
  "BH": "Bahrain",
  "PE": "Peru",
  "BV": "Bouvet_Island",
  "NU": "Niue",
  "PN": "Pitcairn_Islands",
  "CF": "Central_African_Republic",
  "TV": "Tuvalu",
  "CY": "Cyprus",
  "HN": "Honduras",
  "AR": "Argentina",
  "ET": "Ethiopia",
  "JO": "Jordan",
  "LV": "Latvia",
  "CC": "Cocos_Islands",
  "AM": "Armenia",
  "BN": "Brunei",
  "AD": "Andorra",
  "MS": "Montserrat",
  "FO": "Faroe_Islands",
  "SI": "Slovenia",
  "VA": "Vatican_City",
  "TK": "Tokelau",
  "PT": "Portugal",
  "BJ": "Benin",
  "BF": "Burkina_Faso",
  "TR": "Turkey",
  "SK": "Slovakia",
  "FR": "France",
  "SO": "Somalia",
  "TF": "French_Southern_Territories",
  "GR": "Greece",
  "TJ": "Tajikistan",
  "VE": "Venezuela",
  "BI": "Burundi",
  "EE": "Estonia",
  "NZ": "New_Zealand",
  "SR": "Suriname",
  "GD": "Grenada",
  "GP": "Guadeloupe",
  "VG": "British_Virgin_Islands",
  "SJ": "Svalbard_And_Jan_Mayen",
  "MA": "Morocco",
  "NG": "Nigeria",
  "TG": "Togo",
  "NA": "Namibia",
  "MG": "Madagascar",
  "RU": "Russia",
  "CO": "Colombia",
  "IT": "Italy",
  "JE": "Jersey",
  "SS": "South_Sudan",
  "DJ": "Djibouti",
  "CX": "Christmas_Island",
  "PW": "Palau",
  "MD": "Moldova",
  "BB": "Barbados",
  "AQ": "Antarctica",
  "PG": "Papua_New_Guinea",
  "LR": "Liberia",
  "UA": "Ukraine",
  "SN": "Senegal",
  "KG": "Kyrgyzstan",
  "SZ": "Eswatini",
  "HR": "Croatia",
  "DE": "Germany",
  "TC": "Turks_And_Caicos_Islands",
  "GE": "Georgia",
  "AZ": "Azerbaijan",
  "DM": "Dominica",
  "KR": "South_Korea",
  "MP": "Northern_Mariana_Islands",
  "MO": "Macau",
  "GG": "Guernsey",
  "GA": "Gabon",
  "ML": "Mali",
  "RS": "Serbia",
  "EG": "Egypt",
  "RE": "Reunion",
  "IQ": "Iraq",
  "NP": "Nepal",
  "BD": "Bangladesh",
  "KM": "Comoros",
  "MV": "Maldives",
  "UG": "Uganda",
  "VC": "Saint_Vincent_And_The_Grenadines",
  "FM": "Micronesia",
  "AS": "American_Samoa",
  "CK": "Cook_Islands",
  "CD": "Congo_Democratic_Republic",
  "KH": "Cambodia",
  "GW": "Guinea_Bissau",
  "MZ": "Mozambique",
  "AU": "Australia",
  "IS": "Iceland",
  "IR": "Iran",
  "TL": "TimorLeste",
  "OM": "Oman",
  "GS": "South_Georgia_And_South_Sandwich_Islands",
  "AI": "Anguilla",
  "MH": "Marshall_Islands",
  "NC": "New_Caledonia",
  "BZ": "Belize",
  "BT": "Bhutan",
  "TM": "Turkmenistan",
  "ST": "Sao_Tome_And_Principe",
  "GQ": "Equatorial_Guinea",
  "GH": "Ghana",
  "VN": "Vietnam",
  "MM": "Myanmar",
  "BW": "Botswana",
  "TZ": "Tanzania",
  "SE": "Sweden",
  "GB": "UnitedKingdom",
  "ID": "Indonesia",
  "BO": "Bolivia",
  "PR": "Puerto_Rico",
  "LB": "Lebanon",
  "AX": "Aland_Islands",
  "LK": "Sri_Lanka",
  "NL": "Netherlands",
  "EC": "Ecuador",
  "SL": "Sierra_Leone",
  "TD": "Chad",
  "BG": "Bulgaria",
  "TH": "Thailand",
  "AO": "Angola",
  "LY": "Libya",
  "UZ": "Uzbekistan",
  "MN": "Mongolia",
  "SH": "Saint_Helena",
  "XK": "Kosovo",
  "IE": "Ireland",
  "JM": "Jamaica",
  "AE": "United_Arab_Emirates",
  "HK": "Hong_Kong",
  "TW": "Taiwan",
  "BY": "Belarus",
  "CP": "Clipperton_Island",
  "LI": "Liechtenstein",
  "BM": "Bermuda",
  "CV": "CapeVerde",
  "DZ": "Algeria",
  "MR": "Mauritania",
  "HU": "Hungary",
  "SV": "El_Salvador",
  "KY": "Cayman_Islands",
  "GY": "Guyana",
  "SM": "San_Marino",
  "SD": "Sudan",
  "TN": "Tunisia",
  "MW": "Malawi",
  "NO": "Norway",
  "LT": "Lithuania",
  "SC": "Seychelles",
  "MC": "Monaco",
  "LC": "Saint_Lucia",
  "KW": "Kuwait",
  "ZM": "Zambia",
  "LA": "Laos",
  "KZ": "Kazakhstan",
  "ME": "Montenegro",
  "NI": "Nicaragua",
  "UY": "Uruguay",
  "BQ": "Bonaire",
  "FI": "Finland",
  "MU": "Mauritius",
  "LS": "Lesotho",
  "MF": "Saint_Martin",
  "KP": "North_Korea",
  "UM": "US_Minor_Outlying_Islands",
  "CZ": "Czech_Republic",
  "DK": "Denmark",
  "CA": "Canada",
  "BE": "Belgium",
  "FK": "Falkland_Islands",
  "QA": "Qatar",
  "MK": "North_Macedonia",
  "HM": "Heard_Island_And_McDonald_Islands",
  "NF": "Norfolk_Island",
  "PF": "French_Polynesia",
  "TO": "Tonga",
  "CM": "Cameroon",
  "PL": "Poland",
  "GT": "Guatemala",
  "FJ": "Fiji",
  "AL": "Albania",
  "PM": "Saint_Pierre_And_Miquelon",
  "GL": "Greenland",
  "AT": "Austria",
  "PY": "Paraguay",
  "ER": "Eritrea",
  "SA": "Saudi_Arabia",
  "PH": "Philippines",
  "GF": "French_Guiana",
  "MQ": "Martinique",
  "VI": "US_Virgin_Islands",
  "AG": "Antigua_And_Barbuda",
  "GU": "Guam",
  "VU": "Vanuatu",
  "WS": "Samoa",
  "MT": "Malta",
  "US": "United_States",
  "GI": "Gibraltar",
  "CG": "Congo",
  "NE": "Niger",
  "RO": "Romania",
  "IN": "India",
  "CN": "China",
  "AF": "Afghanistan",
  "MY": "Malaysia",
  "IL": "Israel",
  "CL": "Chile",
  "CR": "Costa_Rica",
  "RW": "Rwanda",
  "KN": "Saint_Kitts_And_Nevis",
  "CH": "Switzerland",
  "NR": "Nauru",
  "IM": "Isle_Of_Man"
}
# for key in countries_dict:
#     print(key)
#     print(countries_dict[key])

In [ ]:

country_tile_dict = {} # Dictionary to get the number of countries and the number of tiles
for target_country in countries_dict: # Loop through each country
    
    ONEGEO_API_KEY = "API KEY HERE"  # Replace with your actual One Geo API key
    country_code = target_country # "NG" Example
    country_name = str(countries_dict[target_country]) # "Nigeria" Example
    min_height = 30  # meters
    tile_size = 0.5  # degrees

    # CONNECT TO DUCKDB AND LOAD COUNTRY BOUNDARY FROM OVERTURE
    con = duckdb.connect()
    con.install_extension("spatial")
    con.load_extension("spatial")
    con.sql("LOAD spatial;")
    con.sql("SET s3_region='us-west-2';")

    # Load country boundary and bounding box
    con.sql(f"""
    CREATE OR REPLACE TABLE bounds AS (
        SELECT
            id AS division_id, names.primary, geometry, bbox
        FROM
            read_parquet('s3://overturemaps-us-west-2/release/2025-06-25.0/theme=divisions/type=division_area/*.parquet')
        WHERE
            subtype = 'country'
            AND country = '{country_code}'
    );
    """)

    # Set bounding box variables
    con.sql("SET variable xmin = (SELECT bbox.xmin FROM bounds);") # Min Longitude
    con.sql("SET variable ymin = (SELECT bbox.ymin FROM bounds);") # Min Latitude
    con.sql("SET variable xmax = (SELECT bbox.xmax FROM bounds);") # Max Longitude
    con.sql("SET variable ymax = (SELECT bbox.ymax FROM bounds);") # Max Latitude
    con.sql("SET variable boundary = (SELECT geometry FROM bounds);")

    # Retrieve bounding box values
    bbox_values = con.sql("""
        SELECT 
            getvariable('xmin') AS xmin,
            getvariable('ymin') AS ymin,
            getvariable('xmax') AS xmax,
            getvariable('ymax') AS ymax
    """).fetchall()

    xmin, ymin, xmax, ymax = bbox_values[0]
    print(f"Bounding box: xmin={xmin}, ymin={ymin}, xmax={xmax}, ymax={ymax}")

    # TILE THE BOUNDING BOX 
    tiles = []
    lat = ymin
    while lat < ymax:
        lon = xmin
        while lon < xmax:
            tile = (lon, lat, min(lon + tile_size, xmax), min(lat + tile_size, ymax))
            tiles.append(tile)
            lon += tile_size
        lat += tile_size

    print(f"Generated {len(tiles)} tiles.")

     

    # FILTER TILES THAT INTERSECT WITH COUNTRY GEOMETRY 
    valid_tiles = []
    for tile in tiles:
        west, south, east, north = tile
        # MakeEnvelope creates a rectangle for the tile
        intersects = con.sql(f"""
            SELECT ST_Intersects(
                ST_MakeEnvelope({west}, {south}, {east}, {north}), 
                getvariable('boundary')
            ) AS intersects
        """).fetchone()[0]

        if intersects:
            valid_tiles.append(tile)

    print(f"{len(valid_tiles)} tiles intersect with {country_name}'s boundary.")

    country_tile_dict[country_name] = [] # each country is assigned an empty list as its value to collect all the tiles with features
    # CALL ONE GEO API FOR EACH VALID TILE
    for i, tile in enumerate(valid_tiles):
        west, south, east, north = tile
        bbox_str = f"{west},{south},{east},{north}"
        url = "https://data.onegeo.co/api/"
        params = {
            "token": ONEGEO_API_KEY,
            "bbox": bbox_str,
            "min_height": min_height,
            "types": "commerce,commerce:office,commerce:retail,residence,other,industry:storage",
            "show_sources": "true"
        }

        try:
            response = requests.get(url, params=params)
            if response.status_code == 200:
                data = response.json()
                features = data.get("features", [])
                # create geojson file for tile with features
                if features:
                    country_tile_dict[country_name].append(i+1) # add the tile number that has features to the dictionary to be used for merging files

                    output_file = f"{country_name}_tile_{i+1}.geojson"
                    with open(output_file, "w", encoding="utf-8") as f:
                        json.dump(data, f, indent=2)
                    print(f"Saved: {output_file}")
                else:
                    print(f"No features returned for chunk {i+1}, skipping file.")

        except Exception as ex:
                print(f"Exception for chunk {i+1}: {ex}")

In [ ]:

# Goes through the country_tile dictionary and gets the first geojson file that it finds from the tile_list
for country_name, tiles_list in country_tile_dict.items():
    first_geojson = gpd.read_file(fr"C:\Users\karim\OneDrive\Documents\Propeterra_Data\buildings_by_country\{country_name}_tile_{tiles_list[0]}.geojson")
    # Goes through the rest of the tiles and merges them with each other to get one single geojson file for each country
    for tile in range(1,len(tiles_list)):
        next_geojson = gpd.read_file(fr"C:\Users\karim\OneDrive\Documents\Propeterra_Data\buildings_by_country\{country_name}_tile_{tiles_list[tile]}.geojson")
        result = pd.concat([first_geojson, next_geojson])
        first_geojson = result
    
    output_dir = r'C:\Users\karim\OneDrive\Documents\Propeterra_Data\buildings_by_country' 
    output_file = os.path.join(output_dir, f'{country_name}_buildings.geojson')
    first_geojson = first_geojson.to_crs("EPSG:4087") # Change projection to EPSG:4087 so that it can be visualized in Cesium
    first_geojson.to_file(output_file, driver='GeoJSON')
